In [1]:
import pickle
import os

def bytes_to_str(data):
    return ' '.join(['%.2X' % (x,) for x in data])

In [7]:
dumps = list()
for i in range(5):
    dump = pickle.load(open(f'{i:03d}.dump', 'rb'))
    dumps.append(dump)

In [8]:
exists_in_all_dumps = list()
dump_0 = dumps[0]
for key in dump_0.keys():
    in_all = True
    for dump in dumps[1:]:
        if key not in dump:
            in_all = False
            break
    if in_all:
        exists_in_all_dumps.append(key)
print(f'{len(exists_in_all_dumps)} записей сохраняется по всем дампам')

31 записей сохраняется по всем дампам


In [9]:
const_in_all_dumps = list()
for key in exists_in_all_dumps:
    in_all = True
    for dump in dumps[1:]:
        if dump_0[key] != dump[key]:
            in_all = False
            break
    if in_all:
        const_in_all_dumps.append(key)
print(f'{len(const_in_all_dumps)} записей не меняют содержания по всем дампам:')
for key in const_in_all_dumps:
    print(key)

1 записей не меняют содержания по всем дампам:
('0x6f22600000', 4194304, '[anon:libc_malloc]')


In [10]:
diff_mem = list()
for key in exists_in_all_dumps:
    if key not in const_in_all_dumps:
        diff_mem.append(key)
print('Следующие записи меняются от переговоров к переговорам:')
for key in diff_mem:
    print(key)

Следующие записи меняются от переговоров к переговорам:
('0x6ee2c00000', 41943040, '[anon:libc_malloc]')
('0x6ee6400000', 16777216, '[anon:libc_malloc]')
('0x6ee8a00000', 2097152, '[anon:libc_malloc]')
('0x6ee9000000', 4194304, '[anon:libc_malloc]')
('0x6ee9600000', 4194304, '[anon:libc_malloc]')
('0x6eeaa00000', 27262976, '[anon:libc_malloc]')
('0x6eed400000', 2097152, '[anon:libc_malloc]')
('0x6eeda00000', 4194304, '[anon:libc_malloc]')
('0x6eee800000', 6291456, '[anon:libc_malloc]')
('0x6eef200000', 2097152, '[anon:libc_malloc]')
('0x6eef800000', 2097152, '[anon:libc_malloc]')
('0x6f20200000', 35651584, '[anon:libc_malloc]')
('0x6f22e00000', 2097152, '[anon:libc_malloc]')
('0x6f24200000', 2097152, '[anon:libc_malloc]')
('0x6f24c00000', 4194304, '[anon:libc_malloc]')
('0x6f25400000', 4194304, '[anon:libc_malloc]')
('0x6f27000000', 8388608, '[anon:libc_malloc]')
('0x6f29200000', 4194304, '[anon:libc_malloc]')
('0x6f2a600000', 6291456, '[anon:libc_malloc]')
('0x7011400000', 16777216, '

In [ ]:
poses = dict()
for size in [4, 2, 1, 8]:
    for key in exists_in_all_dumps:
        poses[key] = dict()
        print(key)
        blocks = list()
        for dump in dumps:
            blocks.append(dump[key])
        variants = 0
        print(f'size = {size}')
        poses[key][size] = dict()
        for pos in range(min([len(block) for block in blocks]) - 5*size):
            peaces = set()
            values = set()
            block_values = list()
            for block in blocks:
                block_values.append([])
                peaces.add(block[pos: pos + 5*size])
                for i in range(5):
                    value = int.from_bytes(block[pos + i*size: pos + (i + 1)*size], byteorder="little")
                    block_values[-1].append(value)
                    values.add(value)
            good_blocks = True
            for block in block_values:
                if len(set(block)) < 2 or max(block) > 32:
                    good_blocks = False
                    break
            if good_blocks and len(peaces) == len(blocks) and len(values) <= 4: # В этих переговорах было по 4 товара. Считаем вероятность полного совпадения двух переговоров исчезающе малой.
                poses[key][size][pos] = peaces
                variants += 1
                print(f'{pos}    ({len(set(values))})    {values}')
                for peace, bvalues in zip(peaces, block_values):
                    print(f'    {bytes_to_str(peace)}    ({len(set(bvalues))})    {bvalues}')
        print(f'    {variants} вариантов.')

('0x6ee2c00000', 41943040, '[anon:libc_malloc]')
size = 4
    0 вариантов.
('0x6ee6400000', 16777216, '[anon:libc_malloc]')
size = 4
    0 вариантов.
('0x6ee8a00000', 2097152, '[anon:libc_malloc]')
size = 4
    0 вариантов.
('0x6ee9000000', 4194304, '[anon:libc_malloc]')
size = 4
    0 вариантов.
('0x6ee9600000', 4194304, '[anon:libc_malloc]')
size = 4
    0 вариантов.
('0x6eeaa00000', 27262976, '[anon:libc_malloc]')
size = 4
    0 вариантов.
('0x6eed400000', 2097152, '[anon:libc_malloc]')
size = 4
    0 вариантов.
('0x6eeda00000', 4194304, '[anon:libc_malloc]')
size = 4
    0 вариантов.
('0x6eee800000', 6291456, '[anon:libc_malloc]')
size = 4
6251072    (4)    {1, 2, 3, 4}
    04 00 00 00 01 00 00 00 02 00 00 00 02 00 00 00 03 00 00 00    (3)    [4, 1, 2, 1, 2]
    04 00 00 00 01 00 00 00 02 00 00 00 04 00 00 00 03 00 00 00    (3)    [4, 1, 2, 2, 2]
    04 00 00 00 01 00 00 00 02 00 00 00 01 00 00 00 02 00 00 00    (4)    [4, 1, 2, 2, 3]
    04 00 00 00 01 00 00 00 02 00 00 00 03 00 0